In [10]:
import sys
sys.path.append('../deeprl')
import gym
env = gym.make('CartPole-v1')
import tensorflow as tf
import tflearn as nn

def create_mlp(inputs, n_out):
    net = nn.input_data(placeholder=inputs)
    net = nn.fully_connected(net, 25, activation='relu', weight_decay=0.0001)
    net = nn.dropout(net, 0.3)
    net = nn.fully_connected(net, 25, activation='relu', weight_decay=0.0001)
    net = nn.dropout(net, 0.3)
    net = nn.fully_connected(net, n_out, activation='softmax')
    return net

[2016-11-14 20:03:41,884] Making new env: CartPole-v1


In [14]:
from agents import DPGAgent
from experiment import Experiment

#erase previously created tf graphs
tf.reset_default_graph()

#create a DQNAgent
agent = DPGAgent(create_mlp, env.action_space.n, env.observation_space.shape, alpha=0.001, gamma=0.99,
                    )
exp = Experiment(agent, env)
exp.run_epoch(150000)

[2016-11-14 20:07:10,005] tf.variable_op_scope(values, name, default_name) is deprecated, use tf.variable_scope(name, default_name, values)
[2016-11-14 20:07:10,086] tf.variable_op_scope(values, name, default_name) is deprecated, use tf.variable_scope(name, default_name, values)
[2016-11-14 20:07:10,161] tf.variable_op_scope(values, name, default_name) is deprecated, use tf.variable_scope(name, default_name, values)


episode reward: 58.000000
episode reward: 14.000000
episode reward: 11.000000
episode reward: 14.000000
episode reward: 15.000000
episode reward: 38.000000
episode reward: 14.000000
episode reward: 26.000000
episode reward: 18.000000
episode reward: 16.000000
episode reward: 36.000000
episode reward: 30.000000
episode reward: 16.000000
episode reward: 26.000000
episode reward: 22.000000
episode reward: 34.000000
episode reward: 26.000000
episode reward: 17.000000
episode reward: 19.000000
episode reward: 35.000000
episode reward: 25.000000
episode reward: 44.000000
episode reward: 10.000000
episode reward: 10.000000
episode reward: 24.000000
episode reward: 29.000000
episode reward: 28.000000
episode reward: 12.000000
episode reward: 30.000000
episode reward: 16.000000
episode reward: 13.000000
episode reward: 25.000000
episode reward: 41.000000
episode reward: 29.000000
episode reward: 12.000000
episode reward: 16.000000
episode reward: 16.000000
episode reward: 92.000000
episode rewa

1256